In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Load the data
df_1= pd.read_csv('data1.csv')
df_2= pd.read_csv('data2.csv')
predcit_df = pd.read_csv('predicion.csv')

df_1.head()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df_2.head()

NameError: name 'df_2' is not defined

In [ ]:
# unir los dos dataframes
df = pd.concat([df_1, df_2], axis=0)
df.info()

### Faltan datos en las columnas
#### funder,installer,wpt_name,subvillage,public_meeting,scheme_management,scheme_management,permit,

In [ ]:
df.describe()

In [ ]:
# ver que varables son categoricas y cuales numericas
print(f'Variables Categoricas{df.select_dtypes(include=['object']).columns}')
print(f'Varibales numericas{df.select_dtypes(include=['float64',
                          'int64']).columns}')

In [ ]:
#comprobar valores unicos en todas las columnas
for col in df.columns:
    print(f'{col}: {df[col].unique()}')

### revisando los valores unicos de todas las variables se pueden hacer transformaciones en las variables:

#### waterpoint_type_group,waterpoint_type_group,source_class,source_type,source,quantity_group


In [ ]:
# revisar valores nulos
df.isnull().sum()

#### Se hace la primera transformacion para calcualr la antiguedad del punto de agua y ver si es nuevo, viejo,antiguo o media y asi saber con mas presicion si por ejemplo llos puntos de agua contruidos en ciertos años tienen mas fallas

In [ ]:
# Convertir a datetime
df["date_recorded"] = pd.to_datetime(df["date_recorded"])
df["year_recorded"] = df["date_recorded"].dt.year
df["month_recorded"] = df["date_recorded"].dt.month
df["day_recorded"] = df["date_recorded"].dt.day

predcit_df["date_recorded"] = pd.to_datetime(predcit_df["date_recorded"])
predcit_df["year_recorded"] = predcit_df["date_recorded"].dt.year
predcit_df["month_recorded"] = predcit_df["date_recorded"].dt.month
predcit_df["day_recorded"] = predcit_df["date_recorded"].dt.day

In [ ]:
df["waterpoint_age"] = df["year_recorded"] - df["construction_year"]
df.loc[df["construction_year"] == 0, "waterpoint_age"] = df["waterpoint_age"].median()

predcit_df["waterpoint_age"] = predcit_df["year_recorded"] - predcit_df["construction_year"]
predcit_df.loc[predcit_df["construction_year"] == 0, "waterpoint_age"] = predcit_df["waterpoint_age"].median()

In [ ]:
df.head()

### La segunda transformacion que podemos realizar en agrupar puntos de agua por proximidad esto nos va a permitir saber si los puntos de agua mas alejados del centro tienen mas o menos problemas

In [ ]:
print(df[["longitude", "latitude", "gps_height"]].isnull().sum())

In [ ]:
from sklearn.cluster import KMeans

# Aplicar K-Means con 10 clusters (ajustable)
kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
df["location_cluster"] = kmeans.fit_predict(df[["longitude", "latitude"]])
predcit_df["location_cluster"] = kmeans.predict(predcit_df[["longitude", "latitude"]])

In [ ]:
from geopy.distance import geodesic

# Crear un diccionario con el centro de cada región
region_centers = df.groupby("region")[["latitude", "longitude"]].median()
region_centers2= predcit_df.groupby("region")[["latitude", "longitude"]].median()

# Función para calcular la distancia geográfica
def calcular_distancia(row):
    centro = region_centers.loc[row["region"]]
    return geodesic((row["latitude"], row["longitude"]), (centro["latitude"], centro["longitude"])).km

def calcular_distancia2(row):
    centro = region_centers2.loc[row["region"]]
    return geodesic((row["latitude"], row["longitude"]), (centro["latitude"], centro["longitude"])).km

# Aplicar la función
df["distance_to_region_center"] = df.apply(calcular_distancia, axis=1)
predcit_df["distance_to_region_center"] = predcit_df.apply(calcular_distancia2, axis=1)

In [ ]:
df

In [ ]:
predcit_df

#### La altura (gps_height) puede ser una variable importante si hay patrones de falla en áreas más altas o bajas.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df["gps_height_scaled"] = scaler.fit_transform(df[["gps_height"]])
predcit_df["gps_height_scaled"] = scaler.transform(predcit_df[["gps_height"]])

### Tercera transformacion, convertimos las variables categoricas en numericas

In [ ]:
threshold = 10  # Número mínimo de ocurrencias
# Seleccionar columnas categóricas
cat_cols = df.select_dtypes(include=['object']).columns
cat_cols2 = predcit_df.select_dtypes(include=['object']).columns
for col in cat_cols:
    counts = df[col].value_counts()
    mask = df[col].isin(counts[counts < threshold].index)
    df[col] = np.where(mask, 'Otros', df[col])

for col in cat_cols2:
    counts = predcit_df[col].value_counts()
    mask = predcit_df[col].isin(counts[counts < threshold].index)
    predcit_df[col] = np.where(mask, 'Otros', predcit_df[col])
    

import category_encoders as ce

# Aplicar Binary Encoding
encoder = ce.BinaryEncoder(cols=cat_cols)
encoder2 = ce.BinaryEncoder(cols=cat_cols2)
df_encoded = encoder.fit_transform(df)
df_encoded2 = encoder2.fit_transform(predcit_df)


pd.options.display.max_columns = None
print(df.head())  

In [ ]:
# Rellenar valores nulos con la moda de cada columna
categorical_columns = ["funder", "installer", "scheme_name", "scheme_management",
                       "management", "payment", "water_quality", "quantity", "source", "waterpoint_type"]

for col in categorical_columns:
    df[col].fillna(df[col].mode()[0], inplace=True)
    
for col in categorical_columns:
    predcit_df[col].fillna(predcit_df[col].mode()[0], inplace=True)

In [ ]:
# Función para agrupar categorías poco frecuentes
def reducir_categorias(df, col, threshold=0.01):
    """Agrupa en 'Otros' las categorías que representan menos del threshold (%) de los datos"""
    counts = df[col].value_counts(normalize=True)
    rare_categories = counts[counts < threshold].index
    df[col] = df[col].replace(rare_categories, "Otros")
    return df

def reducir_categorias2(df, col, threshold=0.01):
    """Agrupa en 'Otros' las categorías que representan menos del threshold (%) de los datos"""
    counts = df[col].value_counts(normalize=True)
    rare_categories = counts[counts < threshold].index
    df[col] = df[col].replace(rare_categories, "Otros")
    return df

# Aplicar a las variables con muchas categorías
#for col in ["funder", "installer", "scheme_name"]:
    #df = reducir_categorias(df, col)

In [ ]:
#df = pd.get_dummies(df, columns=["water_quality", "quantity", "source", "waterpoint_type"], drop_first=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Codificar las columnas categóricas
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column].astype(str))

for column in categorical_columns:
    predcit_df[column] = label_encoder.fit_transform(predcit_df[column].astype(str))

In [ ]:
# Contar las categorías únicas por columna categórica
category_counts = df[categorical_columns].nunique()
category_counts2 = predcit_df[categorical_columns].nunique()

# Imprimir las columnas con más de 20 categorías
columns_to_drop = category_counts[category_counts > 20].index
columns_to_drop2 = category_counts2[category_counts2 > 20].index
print(f"Eliminando las siguientes columnas con más de 20 categorías: {columns_to_drop}")
df = df.drop(columns=columns_to_drop)
predcit_df = predcit_df.drop(columns=columns_to_drop2)

### Cuarta Trasformacion Manejo de outlayers

In [ ]:
# Seleccionar columnas numéricas
numerical_columns = df.select_dtypes(include=["int64", "float64"]).columns
numerical_columns2 = predcit_df.select_dtypes(include=["int64", "float64"]).columns

# Función para identificar outliers
def detectar_outliers(df, column):
    Q1 = df[column].quantile(0.25)  # Percentil 25
    Q3 = df[column].quantile(0.75)  # Percentil 75
    IQR = Q3 - Q1  # Rango intercuartil
    lower_bound = Q1 - 1.5 * IQR  # Límite inferior
    upper_bound = Q3 + 1.5 * IQR  # Límite superior
    return df[(df[column] < lower_bound) | (df[column] > upper_bound)]


# Detectar outliers en todas las columnas numéricas
outliers_dict = {col: detectar_outliers(df, col) for col in numerical_columns}
outliers_dict2 = {col: detectar_outliers(predcit_df, col) for col in numerical_columns2}

# Contar cuántos outliers hay por variable
outliers_count = {col: len(outliers_dict[col]) for col in numerical_columns}
print("Cantidad de outliers por variable:", outliers_count)

In [ ]:
df_target = pd.read_csv('objetivo.csv')

df_final = pd.merge(df, df_target, on="id")
df_final

In [ ]:
#ver las variables q no sean numericas
print(f'Variables Categoricas{df_final.select_dtypes(include=["object"]).columns}')

In [ ]:
df_final = df_final.drop(columns=['recorded_by','date_recorded','subvillage'])
predcit_df = predcit_df.drop(columns=['recorded_by','date_recorded','subvillage'])


In [ ]:
# Codificar la variable objetivo 'status_group'
le = LabelEncoder()
df_final['status_group'] = le.fit_transform(df_final['status_group'])

# Realizar One-Hot Encoding para las variables con muchas categorías
df_final = pd.get_dummies(df_final, columns=['wpt_name', 'basin','region', 'lga', 'ward', 
                                              'extraction_type', 'extraction_type_group', 'extraction_type_class', 
                                              'management_group', 'payment_type', 'quality_group', 'quantity_group', 
                                              'source_type', 'source_class', 'waterpoint_type_group'], drop_first=True)

predcit_df = pd.get_dummies(predcit_df, columns=['wpt_name', 'basin', 'region', 'lga', 'ward', 
                                              'extraction_type', 'extraction_type_group', 'extraction_type_class', 
                                              'management_group', 'payment_type', 'quality_group', 'quantity_group', 
                                              'source_type', 'source_class', 'waterpoint_type_group'], drop_first=True)

# Realizar Label Encoding para las variables binarias
le = LabelEncoder()
df_final['public_meeting'] = le.fit_transform(df_final['public_meeting'])
df_final['permit'] = le.fit_transform(df_final['permit'])

predcit_df['public_meeting'] = le.fit_transform(predcit_df['public_meeting'])
predcit_df['permit'] = le.fit_transform(predcit_df['permit'])

In [ ]:
predcit_df

In [ ]:
# Separar las variables independientes (X) y la variable dependiente (y)
X = df_final.drop(columns=["status_group"])  # Reemplaza "target_column" con el nombre de tu columna objetivo
y = df_final["status_group"]  # Variable objetivo

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear el modelo Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
rf.fit(X_train, y_train)

# Realizar predicciones
y_pred_rf = rf.predict(X_test)

# Evaluar el modelo
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))

In [ ]:
X_new = predcit_df.copy()
predictions = rf.predict(X_new)
predcit_df["prediccion"]= predictions
predcit_df.to_csv('resultados_predicciones.csv', index=False)
print("\nPrimeras 5 predicciones:")
print(predcit_df[['id', 'prediccion']].head())